# Symposium


In [3]:
import matplotlib as mpl 
import matplotlib.pyplot as plt 
plt.style.use('seaborn') 
mpl.rcParams['font.family'] = 'serif'
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
%matplotlib inline

In [4]:
mpl.__version__  

'3.0.3'

In [17]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 150)
import scipy as sci
import scipy.stats as scs
import scipy.optimize as sco
import scipy.interpolate as interp
import random as rnd
import debug_control as dbc
import datetime as dt
import json
import businessdate as bdte
import curve_constructor as cv
import interest_rate_base as int_base
import interest_rate_dates as int_date
import interest_rate_capfloor_convenience as capconv
import interest_rate_instruments as int_rate
import interest_rate_discount as int_disc
import interest_rate_swap_convenience as swapconv
import shortrate_model_vasicek as vas

def determine_fnl_caplet(cap, dbg=False):
    max_cplt = cap.reset
    max_key = ''
    for key, val in cap.caplet.items():
        if max_cplt < val.maturity:
            max_cplt = val.maturity
            max_key = val.name
    if dbg:
        print(max_key)
        print(max_cplt, cap.maturity)
    return max_key, max_cplt

filen ="/home/spennington/git/pythoninterestrates/test/data/wk5_psi_fitting.json"
with open(filen, "r") as fp:
    opt2 = json.load(fp)
fp.close()

mdl1 = vas.short_rate_vasicek(0.86, 0.08, 0.01, 0.06, norm_method=capconv.calc_v_norm_1d, dbg=True)

Kappa 0.860000 Theta 0.080000 Sigma 0.010000 r0 0.060000


## Talk Outline
* Starting with Bond Prices -- example (data statistics)
    - Data Review
    - Classification
* __GOAL__: Generate Forward curve (Treasury)
    - Options
    - Bootstrapping
    - Splines
    - Parametric fits
    - PCA
* __GOAL__: Pricing portfolio of bonds (multiple rate dependencies)
    - Derivative rates
    - Refinance incentives
    - Spreads to account for increased credit risk
* __GOAL__: Simulation
    - Option (HJM ... )
    - Vasicek
    - Hull-White
    - LFM
    - LMM
* __Additional Data__:
    - Options Data (Volatility)
    - LIBOR data
* Why Does it matter?
    - Duration metrics assume constant CF in face of changing interest rates
    - LIBOR transition
* Tools
    - QuantLib (C++ & Python)
    - GSL (C)
    - Anaconda (Python)
    - Boost
* Details
* Bibliography

### P1 : A (-1.83)

$$0.06 = \frac{P(0, 0) - P(0, T_{1})}{\sum_{i=1}^{1} \delta_{i} P(0, T_{i})} \quad \implies \quad F(0, 0, 1.0) = 0.06 \implies P(0, T_{1}) = 1/1.06$$
$$0.06 = \frac{P(0, 0) - P(0, T_{2})}{\frac{1}{1.06} + P(0, T_{2})} \implies 0.06(\frac{1}{1.06} + P(0, T_{2})) = 1 - P(0, T_{2}) \implies -\frac{1.0}{1.06} = (-1.06)*P(0,T_{2}) \implies P(0, T_{2}) = \frac{1}{(1.06)^{2}}$$
$$ v_{p}(t=3) = -P(0, 0) + \sum_{i=1}^{2} \delta_{i} P(0, T_{i})*0.05 + P(0, 2.0) = -P(0, 0) + \sum_{i=1}^{2} \delta_{i} P(0, T_{i})*0.06 - \sum_{i=1}^{2} \delta_{i} P(0, T_{i})*(0.01)  + P(0, 2.0)$$
Using the fact the that $-P(0, 0) + \sum_{i=1}^{2}\delta_{i}P(0, T_{i})*0.06 + P(0, 2.0) = 0.0$ and $\delta_{i} = 1.0$, noting the adjustedd sign paying floating. 
$$ \quad \quad v_{p}(t=3) = -0.01\sum_{i=1}^{2} \delta_{i} P(0, T_{i}) $$
Adjusting for notional 100 (0.01*100 =1). and receiver status results:


In [18]:
P1= 1./1.06; P2=1./1.06**2;
P1a=-100; P2a=5/1.06 + 105./(1.06**2)
fnl=-1*(P1 + P2)
print(P1, P2, round(fnl, 2))
print(P1a, P2a, round(P1a+P2a, 2))

0.9433962264150942 0.8899964400142398 -1.83
-100 98.16660733357065 -1.83


### P2 (Correct)
A perpetual bond is a coupon bond with infinite maturity which forever pays annual coupons (at $T_{1}=1$,
$T_{2}=2, …\ldots…)$ of one unit of cash. Assume a flat yield curve $y(0,T) = 8\%$.
Answer the following three questions:

a) What is the price of the perpetual bond at time $t=0$ ? Round your answer to 2 decimal places.
Hint: For any $q \lt 1$, we have $\sum_{k=1}^{\infty}q^{k} = \frac{q}{1-q}$
P2: A
$$ P(t) = \sum_{i=1}^{\infty} e^{-yT_{i}} = \frac{e^{-y}}{1-e^{-y}} = \frac{1}{e^{y}-1}$$
$$ e^{-y} < 1$$


In [19]:
# Price
# print(round(1./0.08,2))
x1=np.exp(0.08)-1.
print(round(np.exp(-0.08)/(1.-np.exp(-0.08)), 2), 1./x1)
# Duration
print(round(np.exp(0.08)/x1, 2))

# Coonvexity

print(round((2.*(x1**(-2))*np.exp(2*0.08) - np.exp(0.08)/x1),2))

12.01 12.006665955663886
13.01
325.34


### P3 (Correct)
b) What is the duration of the perpetual bond at $t=0$ ? Round your answer to 2 decimal places.
### P3: A
$$\frac{dP}{dY} = -(e^{y}-1)^{-2}e^{y} $$
$$ Duration = -\frac{dP}{dY*P} = \frac{(e^{y}-1)^{-2}*e^{y}}{(e^{y} -  1)^{-1}} = (e^{y} - 1)^{-1}*e^{y} $$


### P4 (Correct)
c) What is the convexity of the perpetual bond at $t=0$ ? Round your answer to 2 decimal places.

### P4: A
$$\frac{d^2P}{d^2Y*P} =  \frac{2.*(e^{y} - 1)^{-3}*e^{2y} - (e^{y} - 1)^{-2}*e^{y}}{(e^{y} - 1)^{-1}} = (2.*(e^{y} - 1)^{-2}*e^{2y} - (e^{y} - 1)^{-1}*e^{y})$$

### P5 (Correct)
Consider the UK government bond data in the attached Excel sheet. Coupon payments are semiannual. We assume that coupons can
also be paid on non-business days (e.g. the second coupon of Bond 2 will be paid on 19/7/97, even though this is a Saturday).
Apply the pseudoinverse technique to estimate the vector of discount bond prices $d$ for all cash flow dates, using
the 30/360 daycount convention.
Gilt_data.xls

Answer the following two questions:

a) Use the obtained discount curve to determine the value $V$ at the spot date 4/9/96 of a portfolio with the following cash flows:

| Date (dd/mm/yy) | 27/08/02 | 07/12/05 | 08/09/06 | 13/10/08 |
|:---------------:|:--------:|:--------:|:--------:|:--------:|
| Cash Flow       |     80   |100       |    60    |    250   |

Round your answer to 2 decimal places.
### P5 -- A (219.47)

In [56]:
filen ="../fnl_p5_options.json"
with open(filen, "r") as fp:
    opt2 = json.load(fp)
fp.close()
cf_cpn_mth1 = cv.curve_builder(opt2, method=1, dbg=False)
print(cf_cpn_mth1.results)
columns = ['CF', 'zero', 'maturity','duration']
dates = [bdte.BusinessDate(2002, 8, 27), bdte.BusinessDate(2005, 12, 7), bdte.BusinessDate(2006, 9, 8), bdte.BusinessDate(2008, 10,13)]

cf = pd.DataFrame(np.zeros([len(dates),len(columns)]), columns=columns, index=dates)
simple_cash=[80., 100., 60., 250.]; i=0
for indx, row in cf.iterrows():
    row['CF'] = simple_cash[i]
    row['zero'] = cf_cpn_mth1.zeros.matrix.loc[indx, 'zero']
    row['maturity'] = cf_cpn_mth1.zeros.matrix.loc[indx, 'maturity']
    i = i + 1

print(cf)
print(cf.CF.dot(cf.zero))

        maturity   rate      zero  type  origin       spot     yield   forward  loaded  date_diff
BOND1   0.197222  10.00  0.988762   6.0     1.0   5.762954  5.730449  5.709579     2.0   0.025000
BOND2   1.375000   9.75  0.921479   6.0     1.0   6.197256  5.947317  6.073216     2.0   0.116667
BOND3   2.561111  12.25  0.845396   6.0     1.0   7.140544  6.557694  7.404536     2.0   0.050000
BOND4   3.497222   9.00  0.785571   6.0     1.0   7.805021  6.901021  7.986811     2.0   0.016667
BOND5   5.172222   7.00  0.688587   6.0     1.0   8.743817  7.213801  7.923716     2.0   0.063889
BOND6   5.980556   9.75  0.639642   6.0     1.0   9.420085  7.471646  9.328209     2.0   0.222222
BOND7   9.258333   8.50  0.481013   6.0     1.0  11.653787  7.904892  8.959403     2.0   0.150000
BOND8  10.011111   7.75  0.447859   6.0     1.0  12.314793  8.023862  9.673247     2.0   0.402778
BOND9  12.108333   9.00  0.373299   6.0     1.0  13.864975  8.137994  8.904039     2.0   0.500000
             CF     

### P6
b) Note that the value $V$ of the portfolio in a) is a linear function of the given UK government bond prices
$p_{1},\dots,p_{9}$. For changes $\Delta p_{i}$ in these bond prices, we can therefore write the change $\Delta V$
of $V$ as:
$$\Delta V= \sum_{i=1}^{9} \frac{\partial V}{\partial p_{i}}\Delta p_{i} $$

How many units do you have to buy of Bond 1 to perfectly hedge the portfolio against changes in the price of Bond 1
(everything else being equal)? Round your answer to 2 decimal places.

Hint: Recall that the discount curve obtained by pseudoinverse is given by

$$d = M^{−1}(W^{−1}\Delta^{∗} + e_{1}), \quad \Delta^{∗} = A^{⊤}(AA^{⊤})^{−1}(p−CM^{−1}e_{1})$$

with the same notation as in the lecture and $e_{1}=1, 0 \ldots, 0$. The derivatives of the portfolio value $V$
with respect to the bond prices $p_{i}$ are therefore given by:

$$\frac{\partial V}{\partial p_{1}} \dots \frac{ \partial V}{\partial p_{9}} = C_{port}^{⊤} \frac{\partial d}{\partial p}= C_{port}^{⊤}M^{−1}W^{−1}A^{⊤}(AA^{⊤})^{−1}$$,

where $\frac{\partial d}{\partial p_{ij}}=\frac{\partial d_{i}}{\partial p_{j}}$ and $C_{port}$ is the vector of portfolio cash flows.

### P6: A


In [60]:
A_fnl, M_inverse, W_inverse = cf_cpn_mth1.calc_A_fnl()
# Port construction
port_cf = pd.Series(np.zeros(len(A_fnl)), index=cf_cpn_mth1.cf_matrix.index)
port_cf['2002-08-27'] = 80.0
port_cf['2005-12-07'] = 100.0
port_cf['2006-09-08'] = 60.0
port_cf['2008-10-13'] = 250.0
print(port_cf[port_cf > 0.])
# Derive Calculuation
derivs = port_cf.transpose().dot(M_inverse.dot(W_inverse).dot(A_fnl))
# BOND1
print(cf_cpn_mth1.instruments['BOND1'].cash_flow_df)
mat1 = cf_cpn_mth1.results.loc['BOND1', 'maturity']
cpn = 105. #  mat1*10.0 + 100.0
yld1 = (-1/mat1)*np.log(cf_cpn_mth1.instruments['BOND1'].price/cpn)
dp1_dy = -mat1*cpn*np.exp(-1*yld1*mat1)
print(derivs)
ratio = derivs[0]/dp1_dy
print(mat1, cpn, cf_cpn_mth1.instruments['BOND1'].price, cf_cpn_mth1.instruments['BOND1'].coupon.coupon, yld1)
print(dp1_dy, ratio)
weights = np.linalg.pinv(cf_cpn_mth1.cf_matrix).dot(port_cf)
print(weights)
changes = np.zeros(len(derivs))
changes[0] = 0.5
x1p = derivs.dot(changes) - ratio*dp1_dy
changes[0] = -0.5
x1m = derivs.dot(changes) - ratio*dp1_dy
print(ratio, x1p, x1m)

2002-08-27     80.0
2005-12-07    100.0
2006-09-08     60.0
2008-10-13    250.0
dtype: float64
            maturity  time_diff     CF
1996-11-15  0.197222   0.197222  105.0
[-0.12527352 -0.22072808 -0.23625196 -0.31881763 -0.33374706  0.25853728
  0.44961184  0.31582445  2.23051008]
0.19722222222222222 105.0 103.82 10.0 0.05730449454983884
-20.47561111111111 0.0061181822603293785
[0.         0.         0.         0.         0.         0.74510305
 0.93137009 0.56273807 2.29041096]
0.0061181822603293785 0.06263676033470156 0.18791028100410467


### P7 (Correct)
Consider the following CIR model for the short rate $r(t)$:

$$dr(t)=0.2 (\theta-r(t))dt+0.1\sqrt{r(t)}dW^{*}(t),\quad r(0)=5\%$$
where $W^{*}(t)$ is a standard Brownian motion under the risk-neutral probability measure $\mathbb{Q}$. What is the
mean-reversion level $\theta$ such that the model implied 1-year LIBOR $L(0,1)$ is $5\%$?
Express your answer in percentage points and round to 2 decimal places.
### P7:A (3.78)

In [22]:
# A & B calculating P(0, T=1.)
kappa = 0.2
sigma=0.1
gamma = np.sqrt(kappa**2 + 2.*sigma**2)
r0 = 0.05
t = 1.0
np_diff = (np.exp(t*gamma) - 1.) 
bt = (2.*np_diff)/((gamma + kappa)*np_diff + 2.*gamma)
a1 = np.log((2.*gamma*np.exp(0.5*(gamma + kappa)*t))/((gamma + kappa)*np_diff + 2*gamma))
curr1 = (np.exp(bt*r0))/1.05 
curr2 = np.log(curr1)
curr3 = curr2/a1
theta = curr3*sigma**2/(2.*kappa)
print(a1, curr1, curr2, curr3, theta, round(100*theta, 2))

-0.0023395677162112685 0.9964651713323032 -0.003541090936271416 1.513566336094735 0.03783915840236838 3.78


### P8
Consider the following Vasicek model for the short rate $r(t)$:

$$dr(t)=0.86(0.08-r(t))dt+0.01 dW^{*}(t),\quad r(0)=6\%,$$

where $W^{*}(t)$ is a standard Brownian motion under the risk-neutral probability measure Q. Verify that the random
variable $Z=\log\left(\frac{1}{P(1,2)}\right)$
is normal under the 2-year forward measure $\mathbb{Q}^{2}$. What is the standard deviation of $Z$ under $\mathbb{Q}^{2}$?
Express your answer in basis points and round to 2 decimal places.

### P8:A
Rewriting $Z = \frac{1}{P(1, 2)} = \frac{P(t, 1)}{P(t, 2)}$ which has variance:
$$ \int \limits_{0}^{t} \vert \frac{\sigma}{\kappa}*(e^{-\kappa*(T_{1} - s)} - e^{-\kappa*(T_{2} - s)}) \vert^{2}ds = \int \limits_{0}^{t} \frac{\sigma^2}{\kappa^2}(e^{-\kappa*T_1} - e^{-\kappa*T_2})^2e^{2\kappa*s} ds $$
$$\quad \quad = \frac{\sigma^2}{2*\kappa^3}(e^{-\kappa*T_1} - e^{-\kappa*T_2})^2*e^{2\kappa*s} \vert_{0}^{2} $$ 


In [40]:
r1 = 0.01**2/(2.*0.86**3)
r2=(np.exp(-0.86*1.) - np.exp(-0.86*2))**2
r3=np.exp(4*0.86) - 1.
ans = np.sqrt(r1*r2*r3)
print(r1, r2, r3, ans, round(1e4*ans, 2))

7.860943061617217e-05 0.059582825193663036 30.18695816830946 0.011890703440767515 118.91


### P9 (Correct)
Suppose at time $t=0$ we observe the following forward rates:

| F(0,0,1/4) | F(0,1/4,1/2) | F(0,1/2,3/4) | F(0,3/4,1) | F(0,1,5/4) | F(0,5/4,3/2) | F(0,3/2,7/4) | F(0,7/4,2) |
|:----------:|:------------:|:------------:|:----------:|:----------:|:------------:|:------------:|:----------:|
|  $6\%$    |     $8\%$   |    $9\%$    |   $10\%$  |   $10\%$  |    $10\%$   |     $9\%$   |    $9\%$  |

Consider an at-the-money payer $1\times 1$-swaption where the underlying swap has quarterly fixed payments and notional
$N=1$. Answer the following three questions:

a) Suppose the price of the swaption is equal to $1\%$, what is the implied Black volatility? Express your answer in percentage points and round to 2 decimal places.
### P9:A (30.47)

In [23]:
# Calculating P(0, T_1)
x2 = [6.0, 8.0, 9.0, 10.0, 10.0, 10.0, 9.0, 9.0]
strt_dict = int_date.generate_schedule_dict(start=opt2['start_date'], period='Q',count=(len(x2)+1), date_adjust='none')
x4_disc = int_disc.discount_calculator(x2, data_type=1, dates=strt_dict, dbg=True)
print(x4_disc.matrix)
swap_dict, swap_name = swapconv.generate_swap_dictionary(opt2, max(x4_disc.matrix.index), swapid=1, reference=None, frequency='Q', reset_date='1997-09-04', dbg=True)
swap1 = swapconv.build_swap(swap_name, swap_dict, opt2, dbg=False)
# print(swap1.cash_flow_df)
swap1.calc_swap_strike_forwards(x4_disc, update=True)
# print(swap1.r_swap, swap1.legs[swap1.fixed_loc].coupon.coupon)
# print(swap1.cash_flow_df)
swapt = swapconv.swaption("SWAPTION", swap1.r_swap, swap1, opt2, dbg=False)  # ATM swaption
res = swapt.calc_swaption_implied_volatility(zeros=x4_disc.matrix, price=0.01, mdl='black', dbg=True)
print(round(100*res, 2), swapt.price_swaption(sigma=res, zeros=x4_disc.matrix, mdl='black'))

          maturity  date_diff      zero  forward     yield
19960904      0.00       0.00  1.000000      0.0  0.000000
19961204      0.25       0.25  0.985222      6.0  5.955445
19970304      0.50       0.25  0.965904      8.0  6.938248
19970604      0.75       0.25  0.944649      9.0  7.592246
19970904      1.00       0.25  0.921609     10.0  8.163446
19971204      1.25       0.25  0.899131     10.0  8.506166
19980304      1.50       0.25  0.877201     10.0  8.734646
19980604      1.75       0.25  0.857898      9.0  8.758303
19980904      2.00       0.25  0.839020      9.0  8.776045
Warning -- applying default SWAP definition 
Warning (genrate_swap_dictionary): faulty rates specifcation
sigma 0.00050000 target 0.010000 Value 0.00001647 Diff 0.00998353
sigma 2.00000000 target 0.010000 Value 0.05638259 Diff -0.04638259
sigma 0.35465000 target 0.010000 Value 0.01162413 Diff -0.00162413
sigma 0.30509764 target 0.010000 Value 0.01001357 Diff -0.00001357
sigma 0.30468072 target 0.010000 Valu

### P10 (Correct)
b) Suppose the price of the swaption is equal to $1\%$, what is the implied normal volatility? Express your answer in basis points and round to 2 decimal places.
### P10:A (288.68)

In [24]:
res = swapt.calc_swaption_implied_volatility(zeros=x4_disc.matrix, price=0.01, mdl='bachelier', dbg=True)
print(round(10000*res, 2), swapt.price_swaption(sigma=res, zeros=x4_disc.matrix, mdl='bachelier'))

sigma 0.00050000 target 0.010000 Value 0.00017320 Diff 0.00982680
sigma 2.00000000 target 0.010000 Value 0.69281288 Diff -0.68281288
sigma 0.02886782 target 0.010000 Value 0.01000000 Diff 0.00000000
sigma 0.02886782 target 0.010000 Value 0.01000000 Diff -0.00000000
(0.028867823776661125,       converged: True
           flag: 'converged'
 function_calls: 4
     iterations: 3
           root: 0.028867823776661125)
288.68 0.009999999999999998


### P11
c) Suppose the implied Black volatility of the swaption is equal to $50\%$. What is the corresponding price? Express your answer in percentage points and round to 2 decimal places.
### P11 (1.63)

In [25]:
round(100*swapt.price_swaption(sigma=0.50, zeros=x4_disc.matrix, mdl='black'), 2)

1.63

### P12 (Correct)
Suppose you observe the following swap rates of spot starting swaps with semi-annual fixed leg ($\delta=1/2$):

| Maturity (years)   |  1   |   2  |  3   |   4  |   5  |   6  |   7  |   8  |   9  | 10   |  15  |  20  |  30  |
|:------------------:|:----:|:----:|:----:|:----:|:----:|:----:|:----:|:----:|:----:|:----:|:----:|:----:|:----:|
|  Swap rate ($\%$) | 0.36 | 0.52 | 0.93 | 1.21 | 1.46 | 1.66 | 1.84 | 1.99 | 2.13 | 2.21 | 2.63 | 2.73 | 2.71 |

Use the pseudo-inverse method to estimate a discount curve from the swap rates. Consider now the two-factor Gaussian HJM model with volatility specification

$$ \sigma(t,T)=\left( e^{-\beta_{1}(T-t)}*v_{1} ,\, e^{-\beta_{2}(T-t)}*v_{2}\right)$$

for $v_{1}=0.01$, $v_{2}=0.02$, $\beta_{1}=0.3$, and $\beta_{2}=0.5$. Answer the following three questions:

a) What is the price of an ATM cap with maturity in 30 years, semi-annual cash flows and first reset date in 6 months? Express your answer in percentage points and round to 2 decimal places.
### P12:A (18.39)

In [50]:
filen ="../fnl_p12_options.json"
with open(filen, "r") as fp:
    opt2 = json.load(fp)
fp.close()
cf_mth1 = cv.curve_builder(opt2, method=1, dbg=False)
print(cf_mth1.results)
print(cf_mth1.zeros.matrix)

        maturity  rate      zero  type  origin      spot     yield   forward  loaded  date_diff
SWAP1        1.0  0.36  0.996410   4.0     1.0  0.360318  0.359671  0.353586     2.0        0.5
SWAP2        2.0  0.52  0.989659   4.0     1.0  0.522444  0.519733  0.675021     2.0        0.5
SWAP3        3.0  0.93  0.972422   4.0     1.0  0.945322  0.932166  1.763901     2.0        0.5
SWAP4        4.0  1.21  0.952534   4.0     1.0  1.245784  1.215738  2.078385     2.0        0.5
SWAP5        5.0  1.46  0.929077   4.0     1.0  1.526738  1.471270  2.513647     2.0        0.5
SWAP6        6.0  1.66  0.904249   4.0     1.0  1.764843  1.677516  2.733646     2.0        0.5
SWAP7        7.0  1.84  0.877596   4.0     1.0  1.992520  1.865269  3.023425     2.0        0.5
SWAP8        8.0  1.99  0.850557   4.0     1.0  2.196243  2.023291  3.164460     2.0        0.5
SWAP9        9.0  2.13  0.822378   4.0     1.0  2.399843  2.172836  3.410640     2.0        0.5
SWAP10      10.0  2.21  0.797938   4.0  

In [51]:
mdl2 = vas.short_rate_vasicek(kappa=[0.3, 0.5], theta=0.08, sigma=[0.01, 0.02], r0=0.06,
                             norm_method=capconv.calc_v_norm_nd, dbg=True)
# Strike Calculation (HERE)
reset = bdte.BusinessDate(2013, 4, 3)
maturity = bdte.BusinessDate(2042, 10, 3)
strike = swapconv.calc_swap_strike(reset, maturity, cf_mth1.zeros.matrix, dbg=True)

# print(x4_df[x4_ind].describe())

# Cap Price Calculation
cap30 = int_rate.cap('CAP30', 100*strike, maturity, reset, opt2, frequency='S', dbg=False)
print(cap30.maturity)

price=cap30.price_cap(cf_mth1.zeros.matrix, hjm_model=mdl2)
print(price, round(100*price, 2))

Theta 0.080000 r0 0.060000 
Kappa
[0.3, 0.5]
Sigma
[0.01, 0.02]
calc_swap_strike: num 0.561322 denom 20.281367 strike 0.027677
20421003
params found 
J(0): kappa 0.300000 sigma 0.010000 t 0.000000 t0 0.500000 t1 1.000000
J(0): res1 0.000016 res2 0.583098 intermed 0.000009
J(1): kappa 0.500000 sigma 0.020000 t 0.000000 t0 0.500000 t1 1.000000
J(1): res1 0.000047 res2 0.648721 intermed 0.000031
Di Caplet -- res 0.011977 v_norm 0.000040 d1 1.894180 d2 1.887846 
Caplet: p1 0.029469 p2 0.029397 cpl 0.000072 
params found 
J(0): kappa 0.300000 sigma 0.010000 t 0.000000 t0 1.000000 t1 1.500000
J(0): res1 0.000012 res2 1.370198 intermed 0.000016
J(1): kappa 0.500000 sigma 0.020000 t 0.000000 t0 1.000000 t1 1.500000
J(1): res1 0.000029 res2 1.718282 intermed 0.000049
Di Caplet -- res 0.010315 v_norm 0.000066 d1 1.276652 d2 1.268547 
Caplet: p1 0.101934 p2 0.101542 cpl 0.000392 
params found 
J(0): kappa 0.300000 sigma 0.010000 t 0.000000 t0 1.500000 t1 2.000000
J(0): res1 0.000009 res2 2.432672

### P13 (Correct)
b) What is the implied Black volatility of the cap? Express your answer in percentage points and round to 2 decimal places.
### P13:A 21.85

In [54]:
res_vol_black = cap30.calc_implied_volatility(cf_mth1.zeros.matrix, price=price, hjm_model='black', dbg=True)
round(100*res_vol_black, 2)

sigma 0.00050000 target 0.183919 Value 0.05793505 Diff 0.12598364
sigma 2.00000000 target 0.183919 Value 0.54964207 Diff -0.36572338
sigma 0.51280567 target 0.183919 Value 0.35116633 Diff -0.16724764
sigma 0.22060658 target 0.183919 Value 0.18528352 Diff -0.00136483
sigma 0.21824763 target 0.183919 Value 0.18372272 Diff 0.00019597
sigma 0.21854382 target 0.183919 Value 0.18391881 Diff -0.00000012
sigma 0.21854364 target 0.183919 Value 0.18391869 Diff -0.00000000
sigma 0.21854364 target 0.183919 Value 0.18391869 Diff 0.00000000
sigma 0.21854364 target 0.183919 Value 0.18391869 Diff -0.00000000
(0.21854364237561513,       converged: True
           flag: 'converged'
 function_calls: 9
     iterations: 8
           root: 0.21854364237561513)


21.85

### P14 (Correct)
c) What is the implied normal volatility of the cap? Express your answer in basis points and round to 2 decimal places.
### P14:A 60.73

In [53]:
res_vol_bachelier = cap30.calc_implied_volatility(cf_mth1.zeros.matrix, price=price, hjm_model='bachelier', dbg=True)
round(10000*res_vol_bachelier, 2)

sigma 0.00050000 target 0.183919 Value 0.06080163 Diff 0.12311706
sigma 2.00000000 target 0.183919 Value 54.09733201 Diff -53.91341332
sigma 0.00505567 target 0.183919 Value 0.15822067 Diff 0.02569802
sigma 0.00625683 target 0.183919 Value 0.18859020 Diff -0.00467151
sigma 0.00607206 target 0.183919 Value 0.18388549 Diff 0.00003320
sigma 0.00607337 target 0.183919 Value 0.18391865 Diff 0.00000004
sigma 0.00607337 target 0.183919 Value 0.18391869 Diff -0.00000000
sigma 0.00607337 target 0.183919 Value 0.18391869 Diff 0.00000000
(0.006073369593234005,       converged: True
           flag: 'converged'
 function_calls: 8
     iterations: 7
           root: 0.006073369593234005)


60.73

### Testing --

In [48]:
mdl3 = vas.short_rate_vasicek(kappa=[0.3, 0.5], theta=0.08, sigma=[0.01, 0.02], r0=0.06,
                             norm_method=capconv.calc_v_norm_nd, dbg=False)
filen ="../fnl_p12_options_adjusted.json"
print(round(int_base.convert_annual_yield(0.360, 2), 5))
print(round(int_base.convert_annual_yield(0.520, 2), 5))
print(round(int_base.convert_annual_yield(0.930, 2), 5))
print(round(int_base.convert_annual_yield(1.210, 2), 5))
print(round(int_base.convert_annual_yield(1.460, 2), 5))
print(round(int_base.convert_annual_yield(1.660, 2), 5))
print(round(int_base.convert_annual_yield(1.840, 2), 5))
print(round(int_base.convert_annual_yield(1.990, 2), 5))
print(round(int_base.convert_annual_yield(2.130, 2), 5))
print(round(int_base.convert_annual_yield(2.210, 2), 5))
print(round(int_base.convert_annual_yield(2.600, 2), 5))
print(round(int_base.convert_annual_yield(2.730, 2), 5))
print(round(int_base.convert_annual_yield(2.710, 2), 5))

with open(filen, "r") as fp:
    opt2 = json.load(fp)
fp.close()
cf_mth2 = cv.curve_builder(opt2, method=1, dbg=False)
print(cf_mth2.results)
strike = swapconv.calc_swap_strike(reset, maturity, cf_mth2.zeros.matrix, dbg=False)

# print(x4_df[x4_ind].describe())

# Cap Price Calculation
cap30 = int_rate.cap('CAP30', 100*strike, maturity, reset, opt2, frequency='S', dbg=False)

price=cap30.price_cap(cf_mth2.zeros.matrix, hjm_model=mdl3)
print(cap30.maturity, price)

res_vol_black = cap30.calc_implied_volatility(cf_mth2.zeros.matrix, price=price, hjm_model='black', dbg=True)
res_vol_bachelier = cap30.calc_implied_volatility(cf_mth2.zeros.matrix, price=price, hjm_model='bachelier', dbg=True)
print(round(100*price, 2), round(100*res_vol_black, 2), round(10000*res_vol_bachelier, 2))

0.35968
0.51933
0.92785
1.20636
1.45471
1.65317
1.83161
1.9802
2.11878
2.19792
2.58332
2.71162
2.69188
        maturity     rate      zero  type  origin      spot     yield   forward  loaded  date_diff
SWAP1        1.0  0.35968  0.996413   4.0     1.0  0.359998  0.359351  0.353409     2.0        0.5
SWAP2        2.0  0.51933  0.989672   4.0     1.0  0.521767  0.519064  0.674140     2.0        0.5
SWAP3        3.0  0.92785  0.972486   4.0     1.0  0.943096  0.930001  1.758811     2.0        0.5
SWAP4        4.0  1.20636  0.952674   4.0     1.0  1.241912  1.212050  2.070125     2.0        0.5
SWAP5        5.0  1.45471  0.929328   4.0     1.0  1.520923  1.465867  2.501272     2.0        0.5
SWAP6        6.0  1.65317  0.904630   4.0     1.0  1.757072  1.670488  2.718287     2.0        0.5
SWAP7        7.0  1.83161  0.878131   4.0     1.0  1.982600  1.856561  3.004313     2.0        0.5
SWAP8        8.0  1.98020  0.851256   4.0     1.0  2.184192  2.013037  3.142980     2.0        0.5
SWAP9 